In [27]:
from pyproj import Transformer
import pandas as pd
import numpy as np
import utm

# Potholes Analysis

### Load & Check

In [39]:
pothole_enquiries_2024_5 = pd.read_csv('Pothole_Enquiries_2024.csv')
pothole_enquiries_2024_5.head(2)

,ENQUIRY_TYPE,ENQUIRY_CATEGORY,DATE_RECORDED,DIVISION,CLIENT_OFFICE_NAME,EASTING,NORTHING,APPROVAL_STATUS_NAME,SECTION_NAME
0,Pothole,Public Enquiry,08/07/2024 14:52:00,NORTHERN DIVISION,ANTRIM and NEWTOWNABBEY,299693.580378,393419.216217,Standard - Completed Enquiries,DOONE RD1
1,Pothole,Public Enquiry,21/06/2024 12:02:00,NORTHERN DIVISION,ANTRIM and NEWTOWNABBEY,299593.368350,391364.492782,Standard - Completed Enquiries,BRECART RD1


In [10]:
pothole_enquiries_2024_5.shape

(23016, 9)

In [42]:
pothole_enquiries_2024_5.describe()

,EASTING,NORTHING,lon,lat
count,23016.000000,23016.000000,23016.000000,23016.000000
mean,308098.700910,370305.774791,-6.328752,54.568085
std,37847.114304,31354.166032,0.580453,0.282617
min,0.000000,-2.423449,-10.863443,51.218545
25%,288641.278772,354732.007812,-6.634722,54.429816
50%,313147.511981,369905.718750,-6.253541,54.558453
75%,334557.281250,384464.539212,-5.917788,54.693930
max,366131.062500,444319.234824,-5.437110,55.236448


In [44]:
pothole_enquiries_2024_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ENQUIRY_TYPE          23016 non-null  object 
 1   ENQUIRY_CATEGORY      23016 non-null  object 
 2   DATE_RECORDED         23016 non-null  object 
 3   DIVISION              23016 non-null  object 
 4   CLIENT_OFFICE_NAME    23016 non-null  object 
 5   EASTING               23016 non-null  float64
 6   NORTHING              23016 non-null  float64
 7   APPROVAL_STATUS_NAME  23016 non-null  object 
 8   SECTION_NAME          23016 non-null  object 
 9   lon                   23016 non-null  float64
 10  lat                   23016 non-null  float64
dtypes: float64(4), object(7)
memory usage: 1.9+ MB


In [46]:
pothole_enquiries_2024_5.ENQUIRY_TYPE.value_counts()

ENQUIRY_TYPE
Pothole    23016
Name: count, dtype: int64

In [47]:
pothole_enquiries_2024_5.ENQUIRY_CATEGORY.value_counts()

ENQUIRY_CATEGORY
Public Enquiry                21116
Call Centre Public Enquiry     1890
Complaint                        10
Name: count, dtype: int64

In [48]:
pothole_enquiries_2024_5.DIVISION.value_counts()

DIVISION
SOUTHERN DIVISION    9361
WESTERN DIVISION     5054
NORTHERN DIVISION    5038
EASTERN DIVISION     3552
ALL CLIENTS            11
Name: count, dtype: int64

In [49]:
pothole_enquiries_2024_5.CLIENT_OFFICE_NAME.value_counts()

CLIENT_OFFICE_NAME
ARDS and NORTH DOWN                         3431
CASTLEREAGH and LISBURN                     2316
ARMAGH CITY BANBRIDGE and CRAIGAVON EAST    2311
ANTRIM and NEWTOWNABBEY                     1981
MID and EAST ANTRIM                         1636
ARMAGH CITY BANBRIDGE and CRAIGAVON WEST    1528
FERMANAGH and OMAGH EAST                    1357
NEWRY MOURNE and DOWN WEST                  1356
MID ULSTER SOUTH                            1160
MID ULSTER NORTH                             961
CAUSEWAY COAST and GLENS EAST                937
LONDONDERRY and STRABANE                     902
NEWRY MOURNE and DOWN EAST                   725
BELFAST SOUTH                                679
FERMANAGH and OMAGH WEST                     659
BELFAST NORTH                                556
CAUSEWAY COAST and GLENS WEST                465
?                                             56
Name: count, dtype: int64

In [50]:
pothole_enquiries_2024_5.APPROVAL_STATUS_NAME.value_counts()

APPROVAL_STATUS_NAME
Standard - Completed Enquiries    19975
Standard - New                     2599
Standard - In Progress              441
High Priority - Completed             1
Name: count, dtype: int64

In [51]:
pothole_enquiries_2024_5.SECTION_NAME.value_counts()

SECTION_NAME
CROSSKEYS RD1       124
MOUNTSTEWART RD1    107
BALLYNAHINCH RD3     92
BALLYBLACK RD E1     84
SAINTFIELD RD3       79
                   ... 
LISMAIN ST1           1
GLENSHANE GDS1        1
STEWARTSTOWN AV1      1
ORBY DR1              1
DUNRAVEN PK1          1
Name: count, Length: 6170, dtype: int64

### Convert to Lat/Lng

In [35]:
transformer = Transformer.from_crs("epsg:29903", "epsg:4326", always_xy=True)

easting, northing = 299693.580378, 393419.216217
lon, lat = transformer.transform(easting, northing)
lon, lat

(-6.4513996369441795, 54.778771693036866)

In [40]:

def add_lat_lon(df):
    # Set up transformer from Irish Grid to WGS84
    transformer = Transformer.from_crs("epsg:29903", "epsg:4326", always_xy=True)
    # Apply transformation row-wise
    df["lon"], df["lat"] = zip(*df.apply(
        lambda row: transformer.transform(row["EASTING"], row["NORTHING"]), axis=1
    ))

    return df

add_lat_lon(pothole_enquiries_2024_5)

,ENQUIRY_TYPE,ENQUIRY_CATEGORY,DATE_RECORDED,DIVISION,CLIENT_OFFICE_NAME,EASTING,NORTHING,APPROVAL_STATUS_NAME,SECTION_NAME,lon,lat
0,Pothole,Public Enquiry,08/07/2024 14:52:00,NORTHERN DIVISION,ANTRIM and NEWTOWNABBEY,299693.580378,393419.216217,Standard - Completed Enquiries,DOONE RD1,-6.451400,54.778772
1,Pothole,Public Enquiry,21/06/2024 12:02:00,NORTHERN DIVISION,ANTRIM and NEWTOWNABBEY,299593.368350,391364.492782,Standard - Completed Enquiries,BRECART RD1,-6.453660,54.760341
2,Pothole,Public Enquiry,14/01/2025 14:40:00,NORTHERN DIVISION,ANTRIM and NEWTOWNABBEY,304403.593750,392726.906250,Standard - Completed Enquiries,CLONKEEN RD1,-6.378481,54.771599
3,Pothole,Public Enquiry,04/02/2025 23:16:00,NORTHERN DIVISION,ANTRIM and NEWTOWNABBEY,304324.302980,392798.170892,Standard - Completed Enquiries,CLONKEEN RD1,-6.379687,54.772255
4,Pothole,Public Enquiry,23/02/2025 12:43:00,NORTHERN DIVISION,ANTRIM and NEWTOWNABBEY,304135.583210,393004.102097,Standard - Completed Enquiries,CLONKEEN RD1,-6.382545,54.774144
...,...,...,...,...,...,...,...,...,...,...,...
23011,Pothole,Call Centre Public Enquiry,22/04/2024 14:19:00,WESTERN DIVISION,MID ULSTER NORTH,279815.754940,395083.385076,Standard - Completed Enquiries,GORTNASKEY RD1,-6.759775,54.797267
23012,Pothole,Public Enquiry,24/04/2024 09:37:00,WESTERN DIVISION,MID ULSTER NORTH,279732.549279,394832.377644,Standard - Completed Enquiries,GORTNASKEY RD1,-6.761138,54.795026
23013,Pothole,Public Enquiry,24/04/2024 09:39:00,WESTERN DIVISION,MID ULSTER NORTH,279936.947820,395096.535980,Standard - Completed Enquiries,GORTNASKEY RD1,-6.757888,54.797366
23014,Pothole,Call Centre Public Enquiry,06/02/2025 13:12:00,WESTERN DIVISION,MID ULSTER NORTH,281550.780135,394999.921817,Standard - Completed Enquiries,DUNTIBRYAN RD1,-6.732830,54.796239


In [7]:
pothole_enquiries_2024_5.apply(lambda row: transformer.transform(row['NORTHING'], row['EASTING']), axis=1)

0           (52.59499655299926, -2.09858261954592)
1         (52.594066473143414, -2.128912997829107)
2          (52.63732934727305, -2.108908181364132)
3            (52.636617512426, -2.107853358054016)
4         (52.63492371487582, -2.1048063307013196)
                           ...                    
23011       (52.4163146246091, -2.073714536686857)
23012      (52.41556423523908, -2.077403750936372)
23013     (52.417404275107316, -2.073522999218813)
23014    (52.431911788754896, -2.0749682325653684)
23015    (52.410147574428805, -1.9965863924758913)
Length: 23016, dtype: object

### Clean

In [54]:
unneeded_columns = ["EASTING", "NORTHING", "ENQUIRY_TYPE"]
pothole_enquiries_2024_5.drop(unneeded_columns, axis=1).to_csv("data/pothole_enquiries_2024_5-cleaned.csv", index=False)